In [1]:
import pandas as pd
import numpy as np
from Utilities_fantapalla import *
from joblib import Parallel, delayed
import progressbar
import time
#quotazioni = pd.read_csv ('SCARICA NUOVO FILE E TOGLI I MIGLIORI (8 PORTIERI E 16 PER GLI ALTRI RUOLI).csv')
quotazioni = pd.read_csv('Quotazioni_Fantacalcio_reduced(5-Oct).csv')

In [2]:
portieri = np.array([2, 0, 0, 0])
resto = np.array([0, 6, 6, 4])
num_squadre = 8
teams, team_names = names(num_squadre)
N_cores = 8

In [3]:
rosa_portieri = genera_rose(portieri, num_squadre)

In [4]:
quot_dict_portieri = all_quot_dict(portieri, quotazioni, num_squadre)
quot_dict_resto = all_quot_dict(resto, quotazioni, num_squadre)

In [5]:
def spread(rosa, team_names, quot_dict):
    spread = np.float(np.max(assign_quot(rosa, quot_dict, team_names))-np.min(assign_quot(rosa, quot_dict, team_names)))
    return spread

In [6]:
def find_rosa(struttura_rosa, num_squadre, quotazioni, team_names, iterations = 100):
    rosa = genera_rose(struttura_rosa, num_squadre)
    quot_dict = all_quot_dict(struttura_rosa, quotazioni, num_squadre)
    rose_generate = Parallel(n_jobs = N_cores)(delayed(genera_rose)(struttura_rosa, num_squadre) for i in progressbar.progressbar(range(iterations)))
    all_spread = Parallel(n_jobs = N_cores)(delayed(spread)(rosa, team_names, quot_dict) for rosa in progressbar.progressbar(rose_generate))
    min_ind = np.argmin(all_spread)
    output = pd.DataFrame(data = rose_generate[min_ind], columns = team_names)
    return output

In [34]:
rosa_best_portieri = find_rosa(portieri, num_squadre, quotazioni, team_names, iterations = 5000000)

In [39]:
assign_quot(rosa_best_portieri, quot_dict_portieri, team_names)

,0
Team 1,13.0
Team 2,13.0
Team 3,14.0
Team 4,16.0
Team 5,15.0
Team 6,16.0
Team 7,14.0
Team 8,16.0


In [33]:
rosa_best_resto = find_rosa(resto, num_squadre, quotazioni, team_names, iterations = 50000000)

In [38]:
assign_quot(rosa_best_resto, quot_dict_resto, team_names)

,0
Team 1,191.0
Team 2,192.0
Team 3,192.0
Team 4,190.0
Team 5,191.0
Team 6,190.0
Team 7,190.0
Team 8,190.0


In [51]:
nomi_portieri = id_toName(portieri, quotazioni, rosa_best_portieri, num_squadre, team_names)
nomi_resto = id_toName(resto, quotazioni, rosa_best_resto, num_squadre, team_names)

KeyError: 'FRANCESCO'

In [41]:
struttura_parziale = portieri+resto
nomi_resto.index =  list(range(3,19))
rosa_parziale = pd.concat([nomi_portieri, nomi_resto], axis = 0)
#rosa_parziale.to_pickle('rose_parziali_UFFICIALI.pkl')
FC_colors(rosa_parziale, struttura_parziale)

Team 1,Team 2,Team 3,Team 4,Team 5,Team 6,Team 7,Team 8
ZOET,SPORTIELLO,MUSSO,MONTIPO',CRAGNO,OSPINA,MIRANTE,PERIN
AUDERO,CONSIGLI,RAFAEL,SKORUPSKI,CORDAZ,MERET,SIRIGU,SEPE
ROMERO,IBANEZ,PALOMINO,SPINAZZOLA,BIRAGHI,STRYGER LARSEN,GODIN,DI LORENZO
GHIGLIONE,D'AMBROSIO,N'KOULOU,BRUNO PERES,SKRINIAR,ALEX SANDRO,CHIELLINI,TOLOI
LETIZIA,KUMBULLA,DE SILVESTRI,DJIMSITI,RODRIGUEZ R.,IZZO,PEZZELLA GER.,BRUNO ALVES
GAGLIOLO,MAKSIMOVIC,SMALLING,YOUNG,TOMIYASU,FERRARI G.,CALABRIA,YOSHIDA
MANCINI,GLIK,CALDIROLA,DANILO LAR.,VOJVODA,CACERES,CEPPITELLI,TOLJAN
LUIZ FELIPE,KJAER,COLLEY,GUNTER,RADU,MARIO RUI,LIROLA,ROMAGNOLI
BENTANCUR,ZIELINSKI,RAMIREZ,SENSI,ELMAS,ARTHUR,DE ROON,RABIOT
CALLEJON,SORIANO,KURTIC,LAZZARI,JANKTO,DIAZ B.,PEREYRA,PEREIRO


In [61]:
rosa_parzial_pickle = pd.read_pickle('rose_parziali_UFFICIALI.pkl')
rosa_assegnata = FC_colors(rosa_parzial_pickle, struttura_parziale)

# Ora determiniamo gli accoppiamenti sulla base delle preferenze

In [24]:
import itertools
import random

In [25]:
team_list = list(range(8)) #lista numerica delle squadre
f = ['LUCA', 'PIETRO', 'ENZO', 'NANNI', 'MUSCIOTTO', 'EMILIANO', 'MARIO', 'FRANCESCO']

In [26]:
permutations = list(itertools.permutations(team_list)) #generazione di tutte le possibili peermutazioni
"""
Ogni giocatore scelto contribuisce con un fattore 1000 al peso, quindi se per esempio L ha scelto 2 giocatori dalla
prima squadra e 1 dalla seconda il vettore dei pesi sara':
[2000, 1000, 0, 0, 0, 0, 0, 0]

"""

"\nOgni giocatore scelto contribuisce con un fattore 1000 al peso, quindi se per esempio L ha scelto 2 giocatori dalla\nprima squadra e 1 dalla seconda il vettore dei pesi sara':\n[2000, 1000, 0, 0, 0, 0, 0, 0]\n\n"

In [43]:
#matrice delle preferenze (ESEMPIO)
preferences = np.array([
    [1000, 0,    1000, 0,    0,    0,    0,    1000], #LUCA
    [2000, 1000, 0,    0,    0,    0,    0,    0], #PIETRO
    [0,    1000, 2000, 0,    0,    0,    0,    0], #ENZO
    [0,    0,    0,    1000, 1000, 1000, 0,    0], #NANNI
    [1000, 0,    0,    1000, 0,    1000, 0,    0], #MUSCIOTTO
    [0,    1000, 0,    1000, 0,    1000, 0,    0], #EMILIANO
    [1000, 0,    0,    0,    1000, 0,    1000, 0], #MARIO
    [2000, 1000, 0,    0,    0,    0,    0,    0], #FRANCESCO
])

In [44]:
#assegnazione di una probabilita' ad ogni permutazione sulla base delle preferenze
prob = np.zeros((len(permutations)))
for j in range(len(permutations)):
    prob[j] = 1e200*sum(preferences[i][permutations[j][i]] for i in range(8))


In [45]:
prob_norm = prob/np.sum(prob) #normalizzo la probabilita'
N = 100 #numero di permutazioni con peso piu' alto
ind = np.argpartition(prob_norm, -N)[-N:]
prob2 = np.zeros(len(prob_norm))
prob2[ind] = prob_norm[ind]/np.max(prob_norm)

In [46]:
# una permutazione viene scelta a caso usando la distribuzione di probabilita' assegnata
assignments = random.choices(permutations, weights = prob2, k=1)
dic = {}
for k in progressbar.progressbar(range(len(assignments[0]))):
    dic[f[k]] = ['Team '+str(assignments[0][k]+1)]
    time.sleep(3.)
pd.DataFrame(data=dic)

100% (8 of 8) |##########################| Elapsed Time: 0:00:24 Time:  0:00:24


,LUCA,PIETRO,ENZO,NANNI,MUSCIOTTO,EMILIANO,MARIO,FRANCESCO
0,Team 8,Team 4,Team 3,Team 7,Team 6,Team 2,Team 5,Team 1


In [65]:
rosa_assegnata = rosa_parzial_pickle
rosa_assegnata.columns = team_names

In [64]:
team_names = ['FRANCESCO', 'EMILIANO', 'ENZO', 'PIETRO', 'MARIO', 'MUSCIOTTO', 'NANNI', 'LUCA']

In [67]:
FC_colors(rosa_assegnata,struttura_parziale)

FRANCESCO,EMILIANO,ENZO,PIETRO,MARIO,MUSCIOTTO,NANNI,LUCA
ZOET,SPORTIELLO,MUSSO,MONTIPO',CRAGNO,OSPINA,MIRANTE,PERIN
AUDERO,CONSIGLI,RAFAEL,SKORUPSKI,CORDAZ,MERET,SIRIGU,SEPE
ROMERO,IBANEZ,PALOMINO,SPINAZZOLA,BIRAGHI,STRYGER LARSEN,GODIN,DI LORENZO
GHIGLIONE,D'AMBROSIO,N'KOULOU,BRUNO PERES,SKRINIAR,ALEX SANDRO,CHIELLINI,TOLOI
LETIZIA,KUMBULLA,DE SILVESTRI,DJIMSITI,RODRIGUEZ R.,IZZO,PEZZELLA GER.,BRUNO ALVES
GAGLIOLO,MAKSIMOVIC,SMALLING,YOUNG,TOMIYASU,FERRARI G.,CALABRIA,YOSHIDA
MANCINI,GLIK,CALDIROLA,DANILO LAR.,VOJVODA,CACERES,CEPPITELLI,TOLJAN
LUIZ FELIPE,KJAER,COLLEY,GUNTER,RADU,MARIO RUI,LIROLA,ROMAGNOLI
BENTANCUR,ZIELINSKI,RAMIREZ,SENSI,ELMAS,ARTHUR,DE ROON,RABIOT
CALLEJON,SORIANO,KURTIC,LAZZARI,JANKTO,DIAZ B.,PEREYRA,PEREIRO
